# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
from datetime import date
today = date.today()
CurrentDate = today.strftime("%d/%m/%Y")
CurrentDateFL = today.strftime("%Y%m%d")

CityWeather_df = pd.read_csv(f"C:\\Users\\brand\\Desktop\\python-api-challenge\\WeatherPy\\output_data\\CitiesCurrentWeather{CurrentDateFL}.csv")
CityWeather_df.head()                             

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,escanaba,45.745571,-87.064743,1.09,93,20,2.57,US,1647611651
1,dryden,42.490905,-76.297155,16.05,76,0,1.54,US,1647611652
2,rikitea,-23.122954,-134.968014,26.55,80,13,5.10,PF,1647611653
3,yeppoon,-23.134804,150.743662,24.01,75,97,7.61,AU,1647611654
4,georgetown,6.813743,-58.162447,28.16,82,64,4.43,GY,1647611654


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Drop NaN values
CityWeather_df = CityWeather_df.dropna()

# Store "Lat" and "Lng" into locations
locations = CityWeather_df[["Lat","Lng"]]

#Convert Humidity to float and store
humidity = CityWeather_df["Humidity"]

# Create a Humidity Heatmap layer
fig = gmaps.figure(center=(50.0, -35.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating = False, max_intensity = 300, point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
## Narrow down the CityWeather_df to identify cities with the following weather conditions:
##  - Max Temperature < 27c (80f) but > 21c (70f)
##  - Wind Speed < 16kmph (10mph)
##  - Cloudiness = 0
## Drop null values
IdealCities_df = CityWeather_df.loc[(CityWeather_df["Wind Speed"] < 10) & (CityWeather_df["Cloudiness"] == 0) & \
                                   (CityWeather_df["Max Temp"] < 27) & (CityWeather_df["Max Temp"] > 21)].dropna()


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
## Create hotel_df to only contain City, Lat, Long, and Country columns
hotel_df = IdealCities_df.loc[:,["City", "Lat", "Lng", "Country"]]
hotel_df["Hotel Name"] = ""
hotel_df


,City,Lat,Lng,Country,Hotel Name
76,cape town,-33.928992,18.417396,ZA,
88,port alfred,-33.598184,26.891477,ZA,
94,port elizabeth,-33.961705,25.620752,ZA,
158,pumiao,22.751933,108.490736,CN,
166,faanui,-16.482439,-151.733832,PF,
174,guatemalita,14.117034,-88.599606,HN,
216,east london,-33.019160,27.899857,ZA,
247,codrington,17.640463,-61.825520,AG,
259,alenquer,39.056633,-9.007606,PT,
292,gerash,27.670154,54.137838,IR,


In [9]:
## Using Google Maps API popualte Hotel Name columns with hotel names within 5000 meters of the city's coordinates

import json

# Set base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

# Start for loop to look up hotel names within 5000 meters of city coordinates
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to parameters
    params["location"] = f"{lat},{lng}"

    # make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    #Populates hotel_df "Hotel Name" column with the first hotel extracted from the Google Map API results
    hotel_df.loc[index, "Hotel Name"] = results[0]['name']

Retrieving Results for Index 76: cape town.
{
    "html_attributions": [],
    "next_page_token": "Aap_uEC8_xUNW2Ijoqgh3pUmBdFDGrpJa8X3zHyxMgauGjKrc6fLwscjZqDIPeZbtnmRSCDcUJmYB0gforTKWHk7mcXAAAyhGiRACnDMVozuf7lnyJvstGqPnAZGMdnDwRcjtknJ28Ws-CUwPEcHfZVxKJhEbymOPKfj0p2VKnpCmMNVqS2K34KUsIQ9vMVrOkfMtTe3vLnknsjEsXZCwupCPckh7tPvSYpqIUAjJSAGoeX8gIFveoNtsMwm9q5ombkmXXZ2UHdxIPeAOM4toDqZslZd2sc0KYOk_Nu2OTn2E_k8XN_vtNhrvaKxBtV_OLTf6cFJh8w07pm7j8zVgp3vBigXykGmusxIfhqw8LAlV6KFPpaMbSQwPkjGksoo0WtqOTw1cokmxTnV9ieJU-48ebi5m4huXBec7w2bhMDbyKFBUANudQG0Uw7ClN_drUY",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.9192351,
                    "lng": 18.3858784
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.91793892010728,
                        "lng": 18.38727692989272
                    },
                    "southwest": 

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.6031783,
                    "lng": 26.886978
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.60191127010727,
                        "lng": 26.88814832989272
                    },
                    "southwest": {
                        "lat": -33.60461092989271,
                        "lng": 26.88544867010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Royal St Andrews Hotel",
            "opening_hours": {
                "open_now": true
       

{
    "html_attributions": [],
    "next_page_token": "Aap_uECNLO4xFS_CnbryX7hGlwW30PwLBTtUah49S3hdkHq0kcAvktSiVsqpStIFaJ55s0xQmsHPyuJL5Feg6nD7mvfXGaWod74dXZAt6OAsa1B9LlmVgkfIz4azqzEklEdbIYD7uwuOj6espagkpwyz7wtT3-fDATNwoawZXoBhi4QDBJAV_hm7LASxZ-B_9z1w6VKu-LNTGi4WmBOf08df4ImIBhS2V8jY3cEgObyd0J7QvbzfVLvc7RcsytfaBoc1iNo8PiJkCf83dUUtCQVM3c2q7ueKen5y2faxrmr1EBlJYSk98oAgGmlUUWY6GqmNDks6-UZc_Qnt9VcRQFhISa_xsD357oYF-IS3H3Dn-8FpaS1lh-76hlcn7BL6WTSOPiqiT2etxQi9bfnCWSK46oFeQi6J8ViOiZvef6_X-iq_j3E6LKVKWnC6sPP6aok",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.987501,
                    "lng": 25.6708806
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.98602112010728,
                        "lng": 25.67216262989272
                    },
                    "southwest": {
                        "lat": -33.98872077

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 22.7819556,
                    "lng": 108.4955281
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.78330617989273,
                        "lng": 108.4968857298927
                    },
                    "southwest": {
                        "lat": 22.78060652010728,
                        "lng": 108.4941860701073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Ganquan Hotel",
            "place_id": "ChIJu0iEjL7DszYRzLC8ArHe5Pk",
            "plus_code": {

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 14.0912913,
                    "lng": -88.5611902
                },
                "viewport": {
                    "northeast": {
                        "lat": 14.09264452989272,
                        "lng": -88.55984032010727
                    },
                    "southwest": {
                        "lat": 14.08994487010728,
                        "lng": -88.56253997989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "HOTEL AMERICA",
            "place_id": "ChIJZX7ILKaRZI8RC_RVdAC7iqo",
            "plus_code":

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 17.6372051,
                    "lng": -61.8278427
                },
                "viewport": {
                    "northeast": {
                        "lat": 17.63856067989272,
                        "lng": -61.82642752010727
                    },
                    "southwest": {
                        "lat": 17.63586102010728,
                        "lng": -61.82912717989271
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "TimbukOne",
            "photos": [
                {
                    "height": 4128,
     

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}


IndexError: list index out of range

In [10]:
hotel_df

,City,Lat,Lng,Country,Hotel Name
76,cape town,-33.928992,18.417396,ZA,Mojo Hotel
88,port alfred,-33.598184,26.891477,ZA,Royal St Andrews Hotel
94,port elizabeth,-33.961705,25.620752,ZA,"Radisson Blu Hotel, Port Elizabeth"
158,pumiao,22.751933,108.490736,CN,Ganquan Hotel
166,faanui,-16.482439,-151.733832,PF,InterContinental Bora Bora Resort Thalasso Spa
174,guatemalita,14.117034,-88.599606,HN,HOTEL AMERICA
216,east london,-33.019160,27.899857,ZA,Blue Lagoon Hotel
247,codrington,17.640463,-61.825520,AG,TimbukOne
259,alenquer,39.056633,-9.007606,PT,Quinta Ribeira do Labrador - Lisbon West Wine ...
292,gerash,27.670154,54.137838,IR,


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))